In [2]:
import pandas as pd
import numpy as np
import sklearn
from xgboost import XGBClassifier
import xgboost
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import cross_validate,GridSearchCV

import pickle
from sklearn.utils import shuffle
import json
import os
from xgboost import XGBClassifier
from matplotlib import pyplot
import matplotlib.pylab as plt
%matplotlib inline
import math

In [3]:
#Don't use datetime, Don't use categorical features, xgboost doesn't do those, first one hot encode
featuresToUse = ['visitor_hist_starrating', 'visitor_hist_adr_usd',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'comp1_rate', 'comp1_inv',
                 'mean_prop_starrating',
       'mean_prop_review_score', 'mean_prop_brand_bool',
       'mean_prop_location_score1', 'mean_prop_location_score2',   'mean_prop_log_historical_price','cluster__0','cluster__1','cluster__2','cluster__3','cluster__4','cluster__5','diff_distance_orig','diff_price_usd']
labelToPredict = 'booking_bool'
neededVars = ['srch_id','click_bool']
nameOfModel = "models/10mayClickingOverFitTest"

In [6]:
dftrain = pd.read_csv('../data/80PercentTrainWithMeanNoShuffle.csv', delimiter=",",usecols=featuresToUse+[labelToPredict]+neededVars)
dftrain = dftrain.sample(frac=0.2)
Y = dftrain[labelToPredict]

In [ ]:
param_test1 = {
 'max_depth':range(3,15,2),
 'min_child_weight':range(1,12,2),
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)],
 'gamma':[i/10.0 for i in range(0,5)]
}



gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate = 0.1, n_estimators=120, max_depth=5,
min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
objective= 'reg:logistic', nthread=-1, scale_pos_weight=sum(Y==0)/sum(Y==1), seed=12), 
param_grid = param_test1, scoring='roc_auc',n_jobs=3,iid=False, cv=3,verbose =2)

gsearch1.fit(dftrain[featuresToUse],dftrain[labelToPredict])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

Fitting 3 folds for each of 2880 candidates, totalling 8640 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


In [14]:
gsearch1.cv_results_

C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\utils\de

{'mean_fit_time': array([225.56983967, 255.21629782, 257.57668581, 387.59580064,
        389.38441315, 390.01692839, 539.73805776, 539.98241067,
        539.52982035, 697.71772671, 695.29160032, 697.29982581]),
 'std_fit_time': array([5.25316262, 0.89866657, 1.37081344, 1.41723442, 1.65355861,
        3.07007728, 3.95947194, 3.43512732, 1.50952291, 4.73081391,
        2.83116214, 5.64930892]),
 'mean_score_time': array([1.87859769, 2.18316331, 2.19535475, 2.62139888, 2.69062471,
        2.71096969, 3.23377295, 3.21600866, 3.22898507, 4.03182831,
        3.90218635, 3.93290329]),
 'std_score_time': array([0.15685276, 0.01715887, 0.01483728, 0.02048638, 0.08910174,
        0.08331459, 0.0836278 , 0.02649077, 0.03239451, 0.28624974,
        0.12028085, 0.12103658]),
 'param_max_depth': masked_array(data=[3, 3, 3, 5, 5, 5, 7, 7, 7, 9, 9, 9],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
        fill_value='?',
 